# CNN vs MLP

In [80]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, \
BatchNormalization, GlobalAveragePooling2D, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import random
import tensorflow as tf
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import math

from sklearn.utils import resample
#from utils_gpu import pick_gpu_lowest_memory
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
### Set session
config_1 = tf.ConfigProto()
gpu_fraction_1 = float(os.environ.get('GPU_LIMIT_1', 0.2))
#gpu_fraction_1 = float(os.environ.get('GPU_LIMIT_1', 0.99))
config_1.gpu_options.per_process_gpu_memory_fraction = gpu_fraction_1

sess_1 = tf.Session(config=config_1)
sess_1.run(tf.global_variables_initializer())
K.set_session(sess_1)

### Definitions

In [5]:
def read_data_original():
    #READ DATA FRAME
    (full_x_train, full_y_train), (full_x_test, full_y_test) = cifar10.load_data()
    full_x_train = full_x_train.astype('float32')
    
    #z-score
    mean = np.mean(full_x_train,axis=(0,1,2,3))
    std = np.std(full_x_train,axis=(0,1,2,3))
    full_x_train = (full_x_train-mean)/(std+1e-7)
    full_x_test = (full_x_test-mean)/(std+1e-7)
    
    return (full_x_train, full_y_train), (full_x_test, full_y_test)

### Load data

In [6]:
(full_x_train, full_y_train), (full_x_test, full_y_test) = read_data_original()

num_classes = 10

full_y_train = np_utils.to_categorical(full_y_train,num_classes)
full_y_test = np_utils.to_categorical(full_y_test,num_classes)

##### Define data generator

In [7]:
datagen = ImageDataGenerator(
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=5,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=5,
        channel_shift_range=0.1,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        validation_split=0.0)

In [8]:
datagen.fit(full_x_train)

### Build model

In [46]:
model = Sequential()

model.add(Conv2D(100, kernel_size=3, padding="same", activation="relu", input_shape=(32,32,3)))
model.add(Conv2D(100, kernel_size=3, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.05))

model.add(Conv2D(200, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(200, kernel_size=3, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.05))


model.add(Conv2D(400, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(400, kernel_size=3, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.05))

model.add(Conv2D(800, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(800, kernel_size=3, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(GlobalAveragePooling2D())


model.add(Dropout(0.125))
model.add(Dense(2000))
model.add(Dropout(0.25))
model.add(Dense(10, activation="softmax", name="predictions"))

In [47]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 32, 32, 100)       2800      
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 32, 32, 100)       90100     
_________________________________________________________________
batch_normalization_26 (Batc (None, 32, 32, 100)       400       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 16, 16, 100)       0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 16, 16, 100)       0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 16, 16, 200)       180200    
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 16, 16, 200)      

#### Training

In [48]:
model.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.SGD(lr=0.01),
            metrics=['accuracy'])

In [49]:
batch_size = 200
epochs = 15

In [50]:
callbacks = []

In [51]:
model.fit_generator(datagen.flow(full_x_train, full_y_train, batch_size=batch_size),
                        steps_per_epoch=len(full_x_train)//batch_size,
                        validation_data=(full_x_test, full_y_test),
                        epochs=epochs, verbose=1, workers=20,
                        callbacks=callbacks)

Epoch 1/15
250/250 [==============================] - 60s 241ms/step - loss: 1.6885 - accuracy: 0.4198 - val_loss: 2.9300 - val_accuracy: 0.1000
Epoch 2/15
250/250 [==============================] - 59s 234ms/step - loss: 1.2043 - accuracy: 0.5699 - val_loss: 1.4805 - val_accuracy: 0.4498
Epoch 3/15
250/250 [==============================] - 58s 233ms/step - loss: 1.0171 - accuracy: 0.6377 - val_loss: 0.9595 - val_accuracy: 0.6617
Epoch 4/15
250/250 [==============================] - 59s 234ms/step - loss: 0.9064 - accuracy: 0.6777 - val_loss: 0.8554 - val_accuracy: 0.7085
Epoch 5/15
250/250 [==============================] - 59s 235ms/step - loss: 0.8273 - accuracy: 0.7082 - val_loss: 1.2501 - val_accuracy: 0.6052
Epoch 6/15
250/250 [==============================] - 58s 233ms/step - loss: 0.7704 - accuracy: 0.7291 - val_loss: 0.7395 - val_accuracy: 0.7375
Epoch 7/15
250/250 [==============================] - 59s 235ms/step - loss: 0.7179 - accuracy: 0.7460 - val_loss: 0.7454 - val_ac

In [52]:
scores = model.evaluate(full_x_test, full_y_test, batch_size=batch_size, verbose=1)

10000/10000 [==============================] - 1s 138us/step


In [53]:
print(scores)

[0.5684614914655686, 0.8113999962806702]


### Dense model

In [73]:
model_dense = Sequential()
#model_dense.add(Input(shape=(32,32,3)))
model_dense.add(Flatten(input_shape=(32,32,3)))


model_dense.add(Dropout(0.125))
model_dense.add(Dense(2000))
model_dense.add(Dropout(0.25))
model_dense.add(Dense(10, activation="softmax", name="predictions"))

In [74]:
model_dense.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 3072)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 2000)              6146000   
_________________________________________________________________
dropout_38 (Dropout)         (None, 2000)              0         
_________________________________________________________________
predictions (Dense)          (None, 10)                20010     
Total params: 6,166,010
Trainable params: 6,166,010
Non-trainable params: 0
_________________________________________________________________


In [75]:
model_dense.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.SGD(lr=0.001),
            metrics=['accuracy'])

In [76]:
model_dense.fit_generator(datagen.flow(full_x_train, full_y_train, batch_size=batch_size),
                        steps_per_epoch=len(full_x_train)//batch_size,
                        validation_data=(full_x_test, full_y_test),
                        epochs=epochs, verbose=1, workers=20,
                        callbacks=callbacks)

Epoch 1/15
250/250 [==============================] - 59s 235ms/step - loss: 2.6325 - accuracy: 0.1981 - val_loss: 1.9878 - val_accuracy: 0.3116
Epoch 2/15
250/250 [==============================] - 58s 232ms/step - loss: 2.4055 - accuracy: 0.2398 - val_loss: 1.9059 - val_accuracy: 0.3410
Epoch 3/15
250/250 [==============================] - 58s 233ms/step - loss: 2.3514 - accuracy: 0.2498 - val_loss: 1.8683 - val_accuracy: 0.3470
Epoch 4/15
250/250 [==============================] - 58s 233ms/step - loss: 2.3031 - accuracy: 0.2578 - val_loss: 1.8419 - val_accuracy: 0.3631
Epoch 5/15
250/250 [==============================] - 58s 234ms/step - loss: 2.2680 - accuracy: 0.2613 - val_loss: 1.8249 - val_accuracy: 0.3702
Epoch 6/15
250/250 [==============================] - 59s 234ms/step - loss: 2.2425 - accuracy: 0.2650 - val_loss: 1.8155 - val_accuracy: 0.3688
Epoch 7/15
250/250 [==============================] - 59s 235ms/step - loss: 2.2206 - accuracy: 0.2726 - val_loss: 1.8036 - val_ac

In [77]:
scores_dense = model_dense.evaluate(full_x_test, full_y_test, batch_size=batch_size, verbose=1)

10000/10000 [==============================] - 1s 80us/step


In [79]:
print(scores_dense)

[1.7607048892974853, 0.3935000002384186]


### Conclusions

For images and video processing, advantages of CNN over MLP constitute in:
- ability to extract spacial features (in a "natural" way, with a sliding window)
- shift robustness
- lower complexity (for larger images)
- tranferrability to different image size